In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.init

In [3]:
input=torch.Tensor(1,1,28,28) #배치크기(한번에 로딩하는 이미지 수), 채널, 높이, 너비
print(input.size())
print(input)

torch.Size([1, 1, 28, 28])
tensor([[[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0

In [4]:
conv1=nn.Conv2d(1, 32, 3, padding=1) 
# Conv2d(입력 채널 수, 출력 채널 수, kernal_size(필터의 크기), stride=1, padding=1)
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [5]:
conv2=nn.Conv2d(32, 64, kernel_size=3, padding=1)
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [6]:
pool=nn.MaxPool2d(2)
print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


In [9]:
out1=conv1(input)
out2=pool(out1)
print(out1.size())
print(out2.size())

torch.Size([1, 32, 28, 28])
torch.Size([1, 32, 14, 14])


In [10]:
out3=conv2(out2)
out4=pool(out3)
print(out3.size())
print(out4.size())

torch.Size([1, 64, 14, 14])
torch.Size([1, 64, 7, 7])


In [11]:
out=out4.view(out4.size(0), -1)
print(out.size())

torch.Size([1, 3136])


In [13]:
fc=nn.Linear(3136, 10)
outf=fc(out)
print(outf.size())
print(outf)

torch.Size([1, 10])
tensor([[-0.0417, -0.0051,  0.0823, -0.0831,  0.0042, -0.0126,  0.1115,  0.1041,
          0.0233,  0.0103]], grad_fn=<AddmmBackward0>)


In [15]:
device='cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)

if device=='cuda':
    torch.cuda.manual_seed_all(777)

# device=''
# if torch.cuda.is_available:
#     device='cuda'
# else:
#     device='cpu'

In [14]:
learing_rate=0.001
epochs=15
batch_size=100

In [ ]:
mnist_train=dsets.MNIST(root='MNIST_data',
                        train=True,
                        transform=transforms.ToTensor(),
                        download=True)
mnist_test=dsets.MNIST(root='MNIST_data',
                        train=False,
                        transform=transforms.ToTensor(),
                        download=True)

In [19]:
print(mnist_train)
print(mnist_test)

Dataset MNIST
    Number of datapoints: 60000
    Root location: MNIST_data
    Split: Train
    StandardTransform
Transform: ToTensor()
Dataset MNIST
    Number of datapoints: 10000
    Root location: MNIST_data
    Split: Test
    StandardTransform
Transform: ToTensor()


In [20]:
train_loader=DataLoader(dataset=mnist_train,
                       batch_size=batch_size,
                       shuffle=True,
                       drop_last=False)

test_loader=DataLoader(dataset=mnist_test,
                       batch_size=batch_size,
                       shuffle=True,
                       drop_last=False)

In [23]:
for X, Y in train_loader:
    print(X.size())
    print(Y.size())
    break
for Y,Y in test_loader:
    print(X.size())
    print(Y.size())
    break

torch.Size([100, 1, 28, 28])
torch.Size([100])
torch.Size([100, 1, 28, 28])
torch.Size([100])


In [27]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        #conv layer1
        #image in shape(100, 1, 28, 28)
        # conv -> (?, 32, 28, 28)
        # pool ->(?, 32, 14,14)

        self.layer1=nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        #conv layer1
        #image in shape(?, 32, 14, 14)
        # conv -> (?, 64, 14, 14)
        # pool ->(?, 64, 7, 7)
        self.layer2=nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        #완전 연결층 (100,64*7*7)
        self.fc=nn.Linear(64*7*7, 10, bias=True)
        #완전 연결층 한정으로 가중치 초기화
        nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
            out=self.layer1(x)
            out=self.layer2(out)
            out=out.view(out.size(0),-1)
            out=self.fc(out)
            return out

In [28]:
model=CNN().to(device)
criterion=nn.CrossEntropyLoss().to(device)
optimizer=torch.optim.Adam(model.parameters(), lr=learing_rate)

In [31]:
print(model)
print(list(model.parameters()))

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=3136, out_features=10, bias=True)
)
[Parameter containing:
tensor([[[[-0.2605,  0.0809,  0.1567],
          [ 0.3265,  0.2777, -0.1169],
          [ 0.2477,  0.2583,  0.2690]]],


        [[[-0.1864, -0.0669, -0.1780],
          [ 0.2941,  0.2786,  0.2635],
          [-0.0440, -0.2318, -0.1231]]],


        [[[-0.0819,  0.0778,  0.0237],
          [ 0.3314, -0.0815,  0.3027],
          [ 0.2188,  0.0973,  0.0363]]],


        [[[-0.0865, -0.2869, -0.0754],
          [ 0.1529,  0.2711, -0.3294],
          [-0.3166, -0.0336, -0.1902]]],


        [[[-0.0712, 

In [32]:
train_total_batch=len(train_loader)
test_total_batch=len(test_loader)
print(train_total_batch)
print(test_total_batch)

600
100


In [33]:
for epoch in range(epochs):
    avg_cost=0

    for X, Y in train_loader:
        X=X.to(device)
        Y=Y.to(device)

        optimizer.zero_grad()
        y_hat=model(X)
        cost=criterion(y_hat, Y)
        cost.backward()
        optimizer.step()

        avg_cost+=cost/train_total_batch
    print('Epoc:', epoch, 'cost:', avg_cost)

Epoc: 0 cost: tensor(0.2321, grad_fn=<AddBackward0>)
Epoc: 1 cost: tensor(0.0651, grad_fn=<AddBackward0>)
Epoc: 2 cost: tensor(0.0479, grad_fn=<AddBackward0>)
Epoc: 3 cost: tensor(0.0380, grad_fn=<AddBackward0>)
Epoc: 4 cost: tensor(0.0319, grad_fn=<AddBackward0>)
Epoc: 5 cost: tensor(0.0269, grad_fn=<AddBackward0>)
Epoc: 6 cost: tensor(0.0231, grad_fn=<AddBackward0>)
Epoc: 7 cost: tensor(0.0190, grad_fn=<AddBackward0>)
Epoc: 8 cost: tensor(0.0173, grad_fn=<AddBackward0>)
Epoc: 9 cost: tensor(0.0133, grad_fn=<AddBackward0>)
Epoc: 10 cost: tensor(0.0115, grad_fn=<AddBackward0>)
Epoc: 11 cost: tensor(0.0102, grad_fn=<AddBackward0>)
Epoc: 12 cost: tensor(0.0093, grad_fn=<AddBackward0>)
Epoc: 13 cost: tensor(0.0085, grad_fn=<AddBackward0>)
Epoc: 14 cost: tensor(0.0059, grad_fn=<AddBackward0>)


In [34]:
with torch.no_grad():
    avg_accuracy=0

    for X,Y in test_loader:
        X=X.to(device)
        Y=Y.to(device)
        pred=model(X)
        correct_prde=torch.argmax(pred, -1)==Y
        accuracy=correct_prde.float().sum()
        avg_accuracy+=accuracy
    avg_accuracy=avg_accuracy/test_total_batch
print('Accuracy:', avg_accuracy)

Accuracy: tensor(99.1400)
